In [9]:
import asyncio
import json
import requests


from app.utility.helps import Bob

In [10]:
bob = Bob()
settings = bob.get_settings()
headers = bob.get_context()

In [ ]:
api_url = 'https://api.powerbi.com/v1.0/myorg/gateways'
response = requests.get(url=api_url, headers=headers)
results = response.json()
print(response.status_code)
gateways = list()
for gateway in results['value']:
    gateways.append(gateway.get("id"))


In [ ]:
r = list()
for id in gateways:
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/gateways/{id}/datasources', headers=headers)
    doc_results = response.json()
    r.append(doc_results['value'])   


In [ ]:
gateways = list()

for i in range(0, len(r)):
    if isinstance(r[i], list):
        for j in range(0, len(r[i])):
            gateways.append(r[i][j])
    else:
        gateways.append(r[i])

In [ ]:
gateways

In [ ]:
users = list()

for i in range(0,len(gateways)):
    api_users = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/users'
    response = requests.get(api_users, headers=headers)
    results = response.json()
    results['value'][0]['datasourceId'] = gateways[i]["id"]
    results['value'][0]['gatewayId'] = gateways[i]["gatewayId"]
    users.append(results['value'][0])
    

## Datasource Connectivity  

This only reports back connectivity to the datasource as an error. Also, you need to use the response.text as the JSON is not populated. A successful connection will report back as HTTP 200 while a failed connection will result in HTTP 400

In [ ]:
## This will be used for a template dictionary for a successful connection to the datasource
status_base = {   
    "datasourceId":"",
    "gatewayId":"",
    "error":
    {
        "code":"success",
        "pbi.error":{ 
            "code":"0",
            "parameters":{},
            "details":[],
            "exceptionCulprit":0
        }
    }
}


In [ ]:
status  = list()
for i in range(0,len(gateways)):
    api_status = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/status'
    
    response = requests.get(api_status, headers=headers)
    if response.ok:
        # response.json() is actually empty when it is a success
        
        status_base['datasourceId'] = gateways[i]["id"]
        status_base['gatewayId'] = gateways[i]["gatewayId"]
        status_base.get("error").get("pbi.error").get("details").append({"message":"success","detail":response.status_code})    
        status.append(status_base)
    else:
        results = json.loads(response.text)
        results["datasourceId"] = gateways[i]["id"]
        results["gatewayId"] = gateways[i]["gatewayId"]
        results.get("error").get("pbi.error").get("details").append({"message":"cannot connect","detail":response.status_code})    
        status.append(results)


In [ ]:
datasources = list()

for i in range(0,len(gateways)):
    api_datasource = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}'
    response = requests.get(api_datasource, headers=headers)
    results = response.json()
    results.pop('@odata.context')
    datasources.append(results)


In [ ]:
import smtplib
from email.mime.text import MIMEText

sender_email = 'brcampb@microsoft.com'
receiver_email = 'brandonh.campbell@gmail.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Gmail)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())



sender_email = 'your_email@example.com'
receiver_email = 'recipient_email@example.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Outlook)
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Set up the SMTP server
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'your_email@example.com'
password = 'your_password'

# Create a message
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = 'recipient_email@example.com'

# Send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, [msg['To']], msg.as_string())


In [14]:
# rest_api = "admin/capacities/refreshables"
# GET https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
# htpps://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
rest_api = "https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets&$top=5000"

response = requests.get(url=rest_api, headers=headers)
result = response.json()


In [ ]:
result

In [15]:
# GET https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets/{datasetId}/refreshes
for item in result['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        if dataset['isRefreshable']==True:
            rest_api = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/datasets/{dataset_id}/refreshes"

            response = requests.get(url=rest_api, headers=headers)
            if response.ok:
                results = response.json()
                for result in results['value']:
                    if len(result)>0:
                        print(result)



{'requestId': 'de2143d8-20a8-4064-b508-b80d08153074', 'id': 17140420, 'refreshType': 'Scheduled', 'startTime': '2024-04-02T06:00:06.16Z', 'endTime': '2024-04-02T06:00:34.377Z', 'status': 'Completed', 'refreshAttempts': [{'attemptId': 1, 'startTime': '2024-04-02T06:00:06.2834508Z', 'endTime': '2024-04-02T06:00:32.721383Z', 'type': 'Data'}, {'attemptId': 1, 'startTime': '2024-04-02T06:00:32.9713595Z', 'endTime': '2024-04-02T06:00:33.2369361Z', 'type': 'Query'}]}
{'requestId': '70fd003f-a0b9-4b7c-9e9a-c47cee637e6d', 'id': 17106762, 'refreshType': 'Scheduled', 'startTime': '2024-04-01T06:00:08.077Z', 'endTime': '2024-04-01T06:00:40.03Z', 'status': 'Completed', 'refreshAttempts': [{'attemptId': 1, 'startTime': '2024-04-01T06:00:08.1865199Z', 'endTime': '2024-04-01T06:00:35.249288Z', 'type': 'Data'}, {'attemptId': 1, 'startTime': '2024-04-01T06:00:38.4211943Z', 'endTime': '2024-04-01T06:00:38.702446Z', 'type': 'Query'}]}
{'requestId': '03f43547-95de-4607-ba46-39eabfd5ea9c', 'id': 17072787, '